## Loading Libraries

In [28]:
import pandas as pd
import numpy as np
import torch
import evaluate

## Loading Dataset

In [29]:
df = pd.read_csv('../data/processed/train_sample_processed.csv')

In [30]:
from custom_dataset import GithubDataset

# Create the train dataset
train_dataset = GithubDataset(df)
train_dataset

In [31]:
statuses = np.unique(df['OpenStatus'].values)

id2label = []
label2id = []

for i, val in enumerate(statuses):
  id2label.append({i: val})
  label2id.append({val: i})

## Experimenting With Different Model Architectures

#### COMMENT - IDEAS
We probably should:
- retrain the whole model (probably smaller) with
- better tokenizer - built up from the ground including all the names of the specific tech (languages, frameworks, IDEs, etc.)

### Setting Tokenizer

In [32]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

### Preparing Metrics

In [33]:
accuracy_metric = evaluate.load("accuracy")

In [34]:
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  return accuracy_metric.compute(predictions=predictions, references=labels)

### Creating Models

In [35]:
from transformers import DistilBertModel

model = DistilBertModel.from_pretrained("distilbert-base-uncased")

### Creating Trainer

In [36]:
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
  output_dir="my_awesome_model",
  learning_rate=2e-5,
  per_device_train_batch_size=16,
  per_device_eval_batch_size=16,
  num_train_epochs=2,
  weight_decay=0.01,
  evaluation_strategy="epoch",
  save_strategy="epoch",
  load_best_model_at_end=True,
  push_to_hub=True,
)

# trainer = Trainer(
#   model=model,
#   args=training_args,
#   train_dataset=tokenized_data["train"],
#   eval_dataset=tokenized_data["test"],
#   tokenizer=tokenizer,
#   data_collator=data_collator,
#   compute_metrics=compute_metrics,
# )

# trainer.train()